In [1]:
import polars as pl
import pandas as pd

In [26]:
df_appl_without_null = pl.read_parquet("../../Data/ML_DF/df_appl_without_null.parquet")
data_frame_energy_home = pl.read_parquet("../../Data/ML_DF/df_energy_home.parquet")
df_home_appliances = pl.read_parquet("../../Data/ML_DF/df_home_appliances.parquet")

In [27]:
df_appl_without_null = df_appl_without_null.drop([
    'kettle',
    'dishwasher',
    'microwave',
    'washingmachine',])

In [28]:
df_home_appliances = df_home_appliances.select(
    "homeid_1",
    "date",
    'kettle',
    'dishwasher',
    'microwave',
    'washingmachine',
)

In [29]:
final_df_1 = df_appl_without_null.join(
    data_frame_energy_home.unique(subset=["homeid_1", "date"])
    .select(["homeid_1", "date", "total_power_consumption"]),
    left_on=["homeid_1", "date"],
    right_on=["homeid_1", "date"],
    how="left",
)

In [30]:
final_df_2 = final_df_1.join(
    df_home_appliances.unique(subset=["homeid_1", "date"]),
    left_on=["homeid_1", "date"],
    right_on=["homeid_1", "date"],
    how="left",
)

In [31]:
final_df_1.shape

(2922, 14)

In [32]:
final_df_2.shape

(2922, 18)

In [34]:
final_df_clean = final_df_2.drop_nulls()

In [37]:
final_df_clean.select([
    pl.col(col).is_null().sum().alias(col)
    for col in final_df_2.columns
])

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,hometype_flat,date,smart_automation_binary,total_power_consumption,kettle,dishwasher,microwave,washingmachine
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
final_df_clean = final_df_clean.select(
    'homeid_1',
    'date',

    'num_males',
    'num_females',
    'num_residents',
    'num_children',
    'num_employed',

    'num_with_higher_education',
    'daily_standing_charge_pence',
    'unit_charge_pence_per_kwh',
    
    'n_rooms',
    'hometype_flat',

    'kettle',
    'dishwasher',
    'microwave',
    'washingmachine',
    'smart_automation_binary',
    'total_power_consumption'
)

In [41]:
# usunięcie powtarzających się wierszy - do sprawdzenia
final_df_clean = final_df_clean.filter(pl.col("homeid_1") != 96)

In [43]:
model_df = final_df_clean.drop(["num_females", "daily_standing_charge_pence",
                                "unit_charge_pence_per_kwh", "homeid_1", "date"])

In [44]:
model_df.write_parquet("../../Data/ML_DF/model_df.parquet")

In [45]:
final_df_clean.write_parquet("../../Data/ML_DF/final_df_clean.parquet")